In [1]:
import os
import json
import random 
import pickle

DIR = '../raw_data/2023_1/'
AP_FILE = "APs.pkl"
RP_FILE = "RPs.pkl"
META = "meta.json"

TARGET_APs = []
'''
list of APs' mac address ['mac0', 'mac1', ... ]
'''
TARGET_RPs = {}
'''
dict of RPs {'id1':[rssi_FP1], ... ]
id = room#_spot#_orientation
rssi_FP are integer list
'''
os.system("rm cluster.json")

scanDir = os.scandir(DIR)
jsonFiles = []
for f in scanDir:
    if f.name.endswith(".json"):
        jsonFiles.append(f.name)
print(jsonFiles)

['Intel_3165_B7.json', 'Intel_3165_A6.json', 'Intel_3165_B1.json', 'Intel_3165_C7.json', 'Intel_3165_A7.json', 'pre_mba_C1.json', 'Intel_3165_A4.json', 'pre_mba_C10.json', 'Intel_3165_B2.json', 'Intel_3165_A8.json', 'Intel_3165_A9.json', 'Intel_3165_C5.json', 'Intel_3165_B3.json', 'pre_mba_C11.json', 'Intel_3165_A5.json', 'Intel_3165_B4.json', 'Intel_3165_C2.json', 'Intel_3165_B8.json', 'Intel_3165_A2.json', 'Intel_3165_A10.json', 'Intel_3165_A3.json', 'Intel_3165_C3.json', 'Intel_3165_B5.json']


In [2]:
# generate AP union
index = 0
for fn in jsonFiles:
    with open(DIR + fn, "r") as f:
        records = f.readlines()
    for row in records:
        fp = json.loads(row)["fingerprint"]
        if fp == []:
            continue
        for oneRSSIrecord in fp:
            if oneRSSIrecord[1].lower() not in TARGET_APs:
                TARGET_APs.append(oneRSSIrecord[1].lower())
TARGET_APs.sort()

In [3]:
randString = lambda : ''.join(random.choices("abcdefghijklmnopqrstuvwxyz0123456789", k=5))
# generate FP union
for fn in jsonFiles:
    with open(DIR + fn, "r") as f:
        records = f.readlines()
    room = fn.split(".")[0].split("_")[2]
    for row in records:
        j = json.loads(row)
        id = ".".join([room, j["location"], j["direction"].lower(), str(j["timestamp"]), randString()])
        fp_old = j["fingerprint"]
        if fp_old == []:
            continue
        fp_old_dict = {}
        for rssi in fp_old:
            fp_old_dict[rssi[1].lower()] = rssi[2]
        fp_new = []
        for mac in TARGET_APs:
            if mac in fp_old_dict.keys():
                fp_new.append(fp_old_dict[mac])
            else: fp_new.append(None)
        TARGET_RPs[id] = fp_new
print(len(TARGET_APs))
print(len(TARGET_RPs)) 

# with open(AP_FILE, "wb") as apf:
#     pickle.dump(TARGET_APs, apf)
with open(RP_FILE, "wb") as rpf:
    pickle.dump(TARGET_RPs, rpf)

RP_TRAIN = "train.pkl"
RP_TEST = "test.pkl"
train_dict = {}
test_dict = {}
ratio = 0.9
random.seed()
test_keys = random.sample(list(TARGET_RPs.keys()), int(ratio * len(TARGET_RPs)))
for k in TARGET_RPs.keys():
    if k in test_keys:
        test_dict[k] = TARGET_RPs[k]
    else:
        train_dict[k] = TARGET_RPs[k]

with open(RP_TRAIN, "wb") as rpf:
    pickle.dump(train_dict, rpf)
with open(RP_TEST, "wb") as rpf:
    pickle.dump(test_dict, rpf)

print("train set: {} \t test set: {}".format(len(train_dict), len(test_dict)))

with open(META, "w") as meta:
    j = {
        "num_of_ap" : len(TARGET_APs),
        "num_of_records" : len(TARGET_RPs),
        "ap_list" : TARGET_APs,
        "demo" : {
            "room.pointIndex.orientation.timestamp.random" : "list_of_rssi",
            list(TARGET_RPs.keys())[0] : TARGET_RPs[list(TARGET_RPs.keys())[0]]
        }
    }
    meta.write(json.dumps(j, indent=4))


973
2715
train set: 272 	 test set: 2443
